In [1]:
%pwd

'c:\\Users\\Nandhakumar\\Documents\\End to End Projects\\solar-radiation-prediction\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\Nandhakumar\\Documents\\End to End Projects\\solar-radiation-prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path

In [6]:
from solarRadiation.constants import *
from solarRadiation.utils.common import read_yaml, create_directories

In [7]:

class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from solarRadiation.logging import logger
from solarRadiation.utils.common import get_size
#from solarRadiation.entity import DataIngestionConfig
from pathlib import Path

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts zip file into data directory
        returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
from solarRadiation.config.configuration import ConfigurationManager
from solarRadiation.components.data_ingestion import DataIngestion
from solarRadiation import logger

In [12]:
STAGE_NAME = " Data Ingestion Stage"


class DataIngestionTrainingPipeline:
    def __init__(self) -> None:
        pass

    def main(self):
        config = ConfigurationManager()
        data_ingestion_config = config.get_data_ingestion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)
        data_ingestion.download_file()
        data_ingestion.extract_zip_file()

In [15]:
if __name__ == '__main__':
    try:
        logger.info(f">>>>>>>> statge {STAGE_NAME} started <<<<<<<<")
        obj = DataIngestionTrainingPipeline()
        obj.main()
        logger.info(f">>>>>>>> stage {STAGE_NAME} completed <<<<<<<<\n\nx========x")
    except Exception as e:
        logger.exception(e)
        raise e

[2023-11-25 14:41:41,009: INFO: 4214235063: >>>>>>>> statge  Data Ingestion Stage started <<<<<<<<]
[2023-11-25 14:41:41,034: INFO: common: yaml file config\config.yaml loaded successfully]
[2023-11-25 14:41:41,041: INFO: common: yaml file params.yaml loaded successfully]
[2023-11-25 14:41:41,041: INFO: common: Created directory at artifacts]
[2023-11-25 14:41:41,041: INFO: common: Created directory at artifacts/data_ingestion]
[2023-11-25 14:41:54,431: INFO: 3485779638: artifacts/data_ingestion/data.zip download! with following info: 
X-GUploader-UploadID: ABPtcPpZKJ-1KL7HlP_NhBUVAtTmEbzICI91QWE7mf3M2FA5hTCS5ni8wYv3rs2PfqITdKFcV5Y3-oVq_KhsReY3UdLlS14dux09
Content-Type: application/octet-stream
Content-Security-Policy: sandbox
Content-Security-Policy: default-src 'none'
Content-Security-Policy: frame-ancestors 'none'
X-Content-Security-Policy: sandbox
Cross-Origin-Opener-Policy: same-origin
Cross-Origin-Embedder-Policy: require-corp
Cross-Origin-Resource-Policy: same-site
X-Content-Typ